# Hierarchical Indexing
- Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis.
- It provides a way to work with higher dimension data.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
np.random.uniform(size=9)

array([0.6227876 , 0.16233994, 0.10613757, 0.15429082, 0.57367741,
       0.2716733 , 0.79279439, 0.568418  , 0.75192583])

In [4]:
data = pd.Series(np.random.uniform(size=9),
                index=[list('aaabbccdd'), [1,2,3,1,3,1,2,2,3]])
data

a  1    0.006064
   2    0.684851
   3    0.137637
b  1    0.548987
   3    0.348890
c  1    0.918848
   2    0.249424
d  2    0.114382
   3    0.126006
dtype: float64

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [6]:
data['b']

1    0.548987
3    0.348890
dtype: float64

In [7]:
data['b':'c']

b  1    0.548987
   3    0.348890
c  1    0.918848
   2    0.249424
dtype: float64

In [8]:
data.loc[['b', 'd']]

b  1    0.548987
   3    0.348890
d  2    0.114382
   3    0.126006
dtype: float64

Selection from inner level. Select all values having value 2 from the second index level

In [9]:
data.loc[:, 2]

a    0.684851
c    0.249424
d    0.114382
dtype: float64

In [10]:
data.unstack()

,1,2,3
a,0.006064,0.684851,0.137637
b,0.548987,NaN,0.348890
c,0.918848,0.249424,NaN
d,NaN,0.114382,0.126006


In [11]:
data.unstack().stack()

a  1    0.006064
   2    0.684851
   3    0.137637
b  1    0.548987
   3    0.348890
c  1    0.918848
   2    0.249424
d  2    0.114382
   3    0.126006
dtype: float64

Hierarchical indexing can be done on either axis.

In [12]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [16]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
frame.index.nlevels

2

In [18]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

A `MultiIndex` can be created by itself. For example:

In [19]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                            names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordering and Sorting levels

In [20]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [21]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [22]:
frame.sort_index()

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [23]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [25]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [26]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [27]:
frame.groupby(level='key2').sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
frame.groupby(level='color', axis='columns').sum()

C:\Users\Najee\AppData\Local\Temp\ipykernel_20840\3080437908.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level='color', axis='columns').sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame's columns

In [29]:
frame = pd.DataFrame({'a':range(7), 'b':range(7,0,-1),
                    'c': ['one','one','one','two','two','two','two',],
                    'd':[0,1,2,0,1,2,3,]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [30]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [31]:
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [32]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
